In [49]:
import cv2
import numpy as np
import time

In [ ]:
## fonction draw_rounded_rect pour dessiner un rectangle aux coins arrondis
def draw_rounded_rect(img, pt1, pt2, radius=22, color=(255, 200, 230), thickness=-1):
    x1, y1 = pt1
    x2, y2 = pt2
    radius = int(max(0, min(radius, abs(x2-x1)//2, abs(y2-y1)//2)))
    if thickness < 0:
        cv2.rectangle(img, (x1+radius, y1), (x2-radius, y2), color, -1)
        cv2.rectangle(img, (x1, y1+radius), (x2, y2-radius), color, -1)
        cv2.circle(img, (x1+radius, y1+radius), radius, color, -1)
        cv2.circle(img, (x2-radius, y1+radius), radius, color, -1)
        cv2.circle(img, (x1+radius, y2-radius), radius, color, -1)
        cv2.circle(img, (x2-radius, y2-radius), radius, color, -1)
    else:
        cv2.rectangle(img, pt1, pt2, color, thickness)
        
## fonction overlay_alpha pour superposer une image avec transparence
def overlay_alpha(dst, overlay, alpha=0.75):
    cv2.addWeighted(overlay, alpha, dst, 1-alpha, 0, dst)
 
## fonction mmss pour convertir des secondes en minutes:secondes    
def mmss(seconds: float) -> str:
    s = max(0, int(seconds))
    return f"{s//60:02d}:{s%60:02d}"    

### fonction draw_progress_bar pour dessiner une barre de progression
def draw_progress_bar_bottom(frame, progress, left_text="", right_text=""):
    """Barre en bas de l'écran (hors rectangle)."""
    H, W = frame.shape[:2]
    margin = 24
    bar_h = 22
    y = H - margin - bar_h
    x = margin
    w = W - 2 * margin

    progress = float(np.clip(progress, 0.0, 1.0))

    # shadow
    shadow = frame.copy()
    cv2.rectangle(shadow, (x+4, y+4), (x+w+4, y+bar_h+4), (0,0,0), -1)
    overlay_alpha(frame, shadow, alpha=0.18)

    # background
    overlay = frame.copy()
    cv2.rectangle(overlay, (x, y), (x+w, y+bar_h), (255, 245, 250), -1)
    overlay_alpha(frame, overlay, alpha=0.92)

    # fill
    cv2.rectangle(frame, (x, y), (x + int(w*progress), y+bar_h), (255, 180, 220), -1)

    # border
    cv2.rectangle(frame, (x, y), (x+w, y+bar_h), (60, 30, 60), 2)

    # labels above bar
    if right_text:
        (tw, _), _ = cv2.getTextSize(right_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.putText(frame, right_text, (x+w-tw, y-8),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (120, 40, 120), 2, cv2.LINE_AA)  


def put_fit_text(img, text, org, max_width, font=cv2.FONT_HERSHEY_SIMPLEX,
                 base_scale=1.0, min_scale=0.55, thickness=3, color=(60,30,60)):
    """Ecrit du texte en réduisant la taille si ça dépasse max_width."""
    scale = base_scale
    while scale >= min_scale:
        (tw, th), _ = cv2.getTextSize(text, font, scale, thickness)
        if tw <= max_width:
            cv2.putText(img, text, org, font, scale, color, thickness, cv2.LINE_AA)
            return scale
        scale -= 0.05
    # dernier recours: écrit quand même en petit
    cv2.putText(img, text, org, font, min_scale, color, max(1, thickness-1), cv2.LINE_AA)
    return min_scale

def draw_session_card(frame, session, idx, paused=False, phase="HOLD"):
    """Carte en haut à gauche, plus grande, texte posture auto-fit.
        Carte en haut à gauche.
    - Si phase == "REST" (transition) : affiche la prochaine posture + '(next ✨)'
    - Sinon : affiche la posture en cours
    """
    x, y = 20, 20

    box_w, box_h = 560, 160
    total = len(session)
    # shadow
    overlay = frame.copy()
    draw_rounded_rect(overlay, (x+6, y+6), (x+box_w+6, y+box_h+6),
                      radius=26, color=(0,0,0), thickness=-1)
    overlay_alpha(frame, overlay, alpha=0.20)

    # card
    overlay = frame.copy()
    draw_rounded_rect(overlay, (x, y), (x+box_w, y+box_h),
                      radius=26, color=(255, 235, 245), thickness=-1)
    overlay_alpha(frame, overlay, alpha=0.92)

    # header
    cv2.putText(frame, "MODE SEANCE", (x+24, y+42),
                cv2.FONT_HERSHEY_SIMPLEX, 0.85, (60,30,60), 2, cv2.LINE_AA)
    cv2.putText(frame, f"{idx+1}/{total}", (x+box_w-105, y+42),
                cv2.FONT_HERSHEY_SIMPLEX, 0.85, (60,30,60), 2, cv2.LINE_AA)

    # --- posture text ---
    if phase == "REST" and idx < total - 1:
        posture_text = f"{session[idx+1]['name']} (next)"
    else:
        posture_text = session[idx]["name"]

    max_text_w = box_w - 48
    put_fit_text(
        frame,
        posture_text.upper(),
        (x+24, y+98),
        max_width=max_text_w,
        base_scale=1.05,     ## on peut monter ou descendre cette valeur
        min_scale=0.60,
        thickness=3,
        color=(60,30,60),
    )

    # status
    status = "PAUSE" if paused else ("TRANSITION" if phase == "REST" else "TENIR")
    cv2.putText(frame, f"Etat: {status}", (x+24, y+135),
                cv2.FONT_HERSHEY_SIMPLEX, 0.78, (60,30,60), 2, cv2.LINE_AA)

In [67]:
# ---------------- Session logic ----------------
# Exemple de séance (tu modifies comme tu veux)
SESSION = [
    {"name": "Cobra (Bhujangasana)", "hold_s": 20, "rest_s": 5},
    {"name": "Chien Tete en Bas (Adho Mukha)", "hold_s": 30, "rest_s": 5},
    {"name": "Arbre (Vrksasana)", "hold_s": 20, "rest_s": 0},
]

def run_session(session, cam_index=0):
    cap = cv2.VideoCapture(cam_index)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 540)

    idx = 0
    phase = "HOLD"  # HOLD / REST
    paused = False

    phase_start = time.time()
    pause_start = None
    paused_accum = 0.0

    def phase_duration(i, ph):
        return session[i]["hold_s"] if ph == "HOLD" else session[i].get("rest_s", 0)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        dur = phase_duration(idx, phase)
        now = time.time()

        if paused:
            if pause_start is None:
                pause_start = now
            elapsed = (pause_start - phase_start) - paused_accum
        else:
            if pause_start is not None:
                paused_accum += now - pause_start
                pause_start = None
            elapsed = (now - phase_start) - paused_accum

        # auto-advance
        if not paused and dur > 0 and elapsed >= dur:
            if phase == "HOLD" and session[idx].get("rest_s", 0) > 0:
                phase = "REST"
                phase_start = time.time()
                paused_accum = 0.0
            else:
                phase = "HOLD"
                idx = min(idx + 1, len(session) - 1)
                phase_start = time.time()
                paused_accum = 0.0

        # --- UI ---
        draw_session_card(frame, session, idx, paused=paused, phase=phase)

        progress = 0.0 if dur <= 0 else float(np.clip(elapsed / dur, 0.0, 1.0))
        left = mmss(elapsed)
        right = mmss(max(0, dur - elapsed))
        draw_progress_bar_bottom(frame, progress, left_text=left, right_text=right)

        cv2.putText(
            frame,
            "p:Pause  n:Suivant  b:Precedent  r:Reset  q:Quitter",
            (30, frame.shape[0]-60),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.80,
            (120, 40, 120),
            2,
            cv2.LINE_AA
            )

        cv2.imshow("Yoga Session", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        elif key == ord("p"):
            paused = not paused
        elif key == ord("n"):
            idx = min(idx + 1, len(session) - 1)
            phase = "HOLD"
            phase_start = time.time()
            paused = False
            pause_start = None
            paused_accum = 0.0
        elif key == ord("b"):
            idx = max(idx - 1, 0)
            phase = "HOLD"
            phase_start = time.time()
            paused = False
            pause_start = None
            paused_accum = 0.0
        elif key == ord("r"):
            phase_start = time.time()
            paused = False
            pause_start = None
            paused_accum = 0.0

    cap.release()
    cv2.destroyAllWindows()


In [68]:
run_session(SESSION)